In [1]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [3]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"c:\Users\limye\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\2. Collecting data\250703_Pristine vs O-E\pristine"

# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파라미터 설정
height = 0.001  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)
num_samples = 100000  # 최종 줄이고자 하는 행 수

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=10)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=10)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices]

        # 새 파일 이름 및 저장 경로
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

pristine 0.csv → Peak count: 80395
pristine 1.csv → Peak count: 80566
pristine 10.csv → Peak count: 81579
pristine 15.csv → Peak count: 79567
pristine 2.csv → Peak count: 80407
pristine 20.csv → Peak count: 81859
pristine 25.csv → Peak count: 79988
pristine 3.csv → Peak count: 81234
pristine 30.csv → Peak count: 78522
pristine 4.csv → Peak count: 81769
pristine 5.csv → Peak count: 82416
